In [3]:
import requests
from bs4 import BeautifulSoup
import csv
import re
from tqdm import tqdm
import time
import pandas as pd
from urllib.parse import urlparse, urljoin, urldefrag

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

tqdm.pandas()

In [17]:
league_links = ["https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1",
               "https://www.transfermarkt.com/primera-division/startseite/wettbewerb/ES1",
               "https://www.transfermarkt.com/1-bundesliga/startseite/wettbewerb/L1",
               "https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1",
               "https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1"]

### Getting information on the teams:

In [3]:
def format_currency(value):
    value = value.replace('€', '')
    value = value.replace('Loan fee:', '')
    
    if value[-1] == 'm':
        value = value.replace('m', '')
        return int(float(value) * 1000000)
    
    if value[-2:] == 'bn':
        value = value.replace('bn', '')
        return int(float(value) * 1000000000)

    if value[-1] == '.':
        value = value.replace('.', '')
        if value[-2:] == 'Th':
            value = value.replace('Th', '')
            return int(value) * 1000
    
    return int(value)

def get_league_teams(league_url, year):
    """ Function essentially gets data for teams in the specified link
        for a given year.
        
        Args: 
        - league_link: str, transfermkt link to a league of interest, e.g Premier League
        - year: int, a year specified numerically
        
        Output:
        - Pandas Dataframe containing information of each club for a given year
    """
    teams_list = []

    year_url = urlparse(league_url)._replace(query=f"saison_id={year}").geturl()
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}
    year_html = requests.get(year_url, headers=headers)
    year_soup = BeautifulSoup(year_html.content)
    # print(year_soup.select('.tab-print > .grid-view')[0])
    year_ranking_soup = year_soup.select('.tab-print > .box > .grid-view > .items > tbody')[0].find_all('td')
    
    rank_dict = {}
    
    for i in range(0,len(year_ranking_soup),6):
        team_rank, team_url, _, team_matches, team_pd, team_pts = year_ranking_soup[i:i+6]
        team_url = team_url.find('a')['href']
        team_url = re.sub(r'/spielplan/', r'/startseite/', team_url)

        team_rank = int(team_rank.get_text(strip=True))
        team_league_matches = int(team_matches.get_text(strip=True))
        team_league_pd = int(team_pd.get_text(strip=True))
        team_league_pts = int(team_pts.get_text(strip=True))
        
        rank_dict[team_url] = [team_rank, team_league_matches, team_league_pd, team_league_pts]
        
    year_soup = year_soup.select('.responsive-table > .grid-view > .items > tbody')[0]
    for idx, cells in tqdm(enumerate(year_soup.find_all(class_=re.compile("^(even|odd)$")))):
        td_tags = cells.find_all("td")
        team_url = td_tags[1].find('a')['href']
        team_rank, team_league_matches, team_league_pd, team_league_pts = rank_dict[team_url]
        team_url = urljoin(league_url, team_url)
        team_html = requests.get(team_url, headers=headers)
        team_soup = BeautifulSoup(team_html.content, "lxml")

        team_manager = team_soup.select('.container-hauptinfo')[0].get_text(strip=True)
        team_transfer_income = format_currency(team_soup.select('.transfer-record__total--positive')[0].get_text(strip=True))
        team_transfer_expense = format_currency(team_soup.select('.transfer-record__total--negative')[0].get_text(strip=True))
        team_transfer_net = team_transfer_income - team_transfer_expense

        team_name = td_tags[1].get_text(strip=True)
        team_avg_age = float(td_tags[3].get_text(strip=True))
        team_foreigners = int(td_tags[4].get_text(strip=True))
        team_avg_val = td_tags[5].get_text(strip=True)
        team_total_val = td_tags[6].get_text(strip=True)


        team = {
                'team_name': team_name,
                'team_url': team_url,
                'team_manager': team_manager,
                'team_league_matches': team_league_matches,
                'team_league_pd': team_league_pd,
                'team_league_pts': team_league_pts,
                'team_avg_age': team_avg_age,
                'team_foreigners': team_foreigners,
                'team_avg_val': format_currency(team_avg_val),
                'team_total_val': format_currency(team_total_val),
                'team_transfer_income': team_transfer_income,
                'team_transfer_expense': team_transfer_expense,
                'team_transfer_net': team_transfer_net,
                'team_rank': team_rank,
                'season': year
                }
        teams_list.append(team)

    return teams_list



In [ ]:
teams = []
start_year = 2018
end_year = 2021

for league in tqdm(league_links):
    for year in tqdm(range(start_year, end_year+1)):
        teams.extend(get_league_teams(league, year))

teams_df = pd.DataFrame(teams)
teams_df.to_csv("majorleagues_teams_20182021.csv", index=False)

### Getting information of player stats per year:

In [4]:
combined = pd.read_csv("majorleagues_20192021.csv")

In [5]:
combined_stats = combined[["player_name", "player_stats_url"]]

In [6]:
player_stats = []

def int_string(string):
    string = re.sub(r"[\-\.\']", "", string)
    if not string:
        return 0
    else:
        return int(string)
    
def get_player_stats(stats_url, year):
    stats_year_url = urlparse(stats_url)._replace(query=f"saison={year}").geturl()
    # print(stats_year_url)
    headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.51 Safari/537.36"}
    stats_html = requests.get(stats_year_url, headers=headers)
    stats_soup = BeautifulSoup(stats_html.content)
    try:
        res = stats_soup.select('.responsive-table > .grid-view > .items > tfoot')[0].find_all('td')[2:]
        res = list(map(int_string, [x.text for x in res]))
        games, goals_scored, goals_concede, clean_sheet, assists, yellow, second_yellow, red, mins = [0] * 9
        if len(res) == 7:
            games, goals_scored, assists, yellow, second_yellow, red, mins = res
        else:
            games, goals_scored, yellow, second_yellow, red, goals_concede, clean_sheet, mins = res
            
    except:
        res = None
        
    return [games, goals_scored, goals_concede, clean_sheet, assists, yellow, second_yellow, red, mins] if res else [0] * 9
    
for name, stats_url in tqdm(combined_stats.values):
    for year in range(2018, 2022):
        res = get_player_stats(stats_url, year)
        player_stats.append([name, year] + res)


100%|███████████████████████████████████████| 642/642 [2:03:22<00:00, 11.53s/it]


In [8]:
stats_df

,0,1,2,3,4,5,6,7,8,9,10
0,Jack Grealish,2018,35,6,0,0,8,5,0,0,3031
1,Jack Grealish,2019,41,10,0,0,8,8,0,0,3550
2,Jack Grealish,2020,27,7,0,0,12,5,0,0,2275
3,Jack Grealish,2021,29,4,0,0,3,2,0,0,2110
4,Romelu Lukaku,2018,45,15,0,0,4,5,0,0,3001
...,...,...,...,...,...,...,...,...,...,...,...
2563,Rogério,2021,19,0,0,0,2,5,0,0,1554
2564,Marc Cucurella,2018,33,2,0,0,2,6,0,0,2536
2565,Marc Cucurella,2019,46,1,0,0,6,10,0,0,3749
2566,Marc Cucurella,2020,39,3,0,0,2,9,0,0,3324


In [9]:
stats_df = pd.DataFrame(player_stats)
stats_df.columns = ['player_name', 'season', 'games', 'goals_scored', 'goals_concede', 'clean_sheet', 'assists', 'yellow', 'second_yellow', 'red', 'mins']
stats_df.to_csv("majorleagues_stats.csv", index=False)